In [ ]:
import ixmp
import yaml
import pyam
import nomenclature

In [ ]:
ar6_public = ixmp.Platform("ar6")
ar6_submission = ixmp.Platform("ar6-submission")

In [ ]:
ar6_public_scenario_list = ar6_public.scenario_list()
ar6_submission_scenario_list = ar6_submission.scenario_list()

In [ ]:
with open("region_rename.yaml") as f:
    region_rename_mapping = yaml.safe_load(f)

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
cdlinks_scenarios = [
    'CD-LINKS_NoPolicy',
    'CD-LINKS_INDCi',
    'CD-LINKS_INDC2030i_1600',
    'CD-LINKS_INDC2030i_1000',
    'CD-LINKS_INDC2030i_400',
    'CD-LINKS_NDC2030i_1000',
    'CD-LINKS_NPi',
    'CD-LINKS_NPi2020_1600',
    'CD-LINKS_NPi2020_1000',
    'CD-LINKS_NPi2020_400',
 ]

_scenarios = ar6_public_scenario_list[[s in cdlinks_scenarios for s in ar6_public_scenario_list.scenario]]

In [ ]:
scenario_selection = list()
for i, row in _scenarios.iterrows():
    if row["model"] in ["AIM/CGE 2.1", "MESSAGEix-GLOBIOM 1.0", "IMAGE 3.0.1", "REMIND-MAgPIE 1.7-3.0", "WITCH-GLOBIOM 4.4"]:
        scenario_selection.append((row["model"], row["scenario"]))

In [ ]:
scenario_selection

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=ar6_submission, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.rename(region=region_rename_mapping, inplace=True)

In [ ]:
df.rename(region={
    "R5OECD90+EU": "OECD & EU (R5)",
    "R5ASIA": "Asia (R5)",
    "R5REF": "Reforming Economies (R5)",
    "R5MAF": "Middle East & Africa (R5)",
    "R5LAM": "Latin America (R5)",
}, inplace=True)

In [ ]:
# REMIND 1.7

remind_eu_region_rename_mapping = dict([
    (i, i.replace("REMIND-MAgPIE 1.7-3.0", "REMIND 1.7")) for i in df.region if "REMIND" in i
])
df.rename(region=remind_eu_region_rename_mapping, inplace=True)

remind_region_rename_mapping = {
    "REMIND 2.0|Canada, Australia and New Zealand":
      "REMIND 2.0|Canada, Australia, New Zealand",
    "REMIND 2.0|China":
      "REMIND 2.0|China and Taiwan",
    "REMIND 2.0|Russia and Reforming Ecomonies":
      "REMIND 2.0|Russia and Reforming Economies",
    "REMIND 2.0|Middle East, North Africa, Central Asia":
      "REMIND 2.0|Middle East and North Africa",
    "REMIND 2.0|Sub-saharan Africa":
     "REMIND 2.0|Sub-Saharan Africa",
    "REMIND 2.0|Other Asian countries":
      "REMIND 2.0|Other Asia", 
    "REMIND 2.0|European Union Center East countries":
      "REMIND 2.0|EU Center-East Europe",
    "REMIND 2.0|European Union Center South":
      "REMIND 2.0|EU Center-South Europe",
    "REMIND 2.0|European Union North Center countries":
      "REMIND 2.0|EU North-Center Europe",
    "REMIND 2.0|European Union South Center countries":
      "REMIND 2.0|EU South-Center Europe",
    "REMIND 2.0|European Union South West countries":
      "REMIND 2.0|EU South-West Europe",
    "REMIND 1.7|Sub-saharan Africa (excl. South Africa)":
      "REMIND 1.7|Sub-Saharan Africa (excl. South Africa)",
    "REMIND 1.7|other Asia":
      "REMIND 1.7|Other Asia",
    "REMIND 1.7|rest of the World (Canada, South Africa, NZ, Australia, Non-EU27 Europe)":
      "REMIND 1.7|Rest of the World",
}
df.rename(region=remind_region_rename_mapping, inplace=True)

In [ ]:
df.region

In [ ]:
df.filter(region=[
    "China", "India", "Japan", "Brazil", "Mexico", 
    "United States of America", "European Union (28 member countries)",
    "REMIND 1.7*"
], keep=False, inplace=True)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
df.rename(unit={
    "billion Int$2010/yr": "billion US$2010/yr",
    "bn vkm/yr": "billion vkm/yr",
    "bn pkm/yr": "billion pkm/yr",
    "bn tkm/yr": "billion tkm/yr",
    "bn m2/yr": "billion m2/yr",
    "Mt/year": "Mt/yr",
}, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)
    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(df.data)
    scen.commit(f"Migrated from the IPCC AR6 database")
    scen.set_as_default()

    scen.set_meta({"Source": f"CD-LINKS (via IPCC AR6)"})
    
    print(f"Model {_model} - Scenario {_scenario}" )